In [98]:
import pandas as pd
import numpy as np

In [99]:
teams = pd.read_csv('DataFiles/Teams.csv')
reg_season = pd.read_csv('DataFiles/RegularSeasonDetailedResults.csv')
reg_season_2018 = pd.read_csv('PrelimData2018/RegularSeasonDetailedResults_Prelim2018.csv')
reg_season = reg_season.append(reg_season_2018, i)


team_ids = teams.TeamID.tolist()
stat_categories = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']

In [100]:
def swap_locations(locations):
    swapped_locations = []
    for l in locations:
        if l == 'A':
            swapped_locations.append('H')
        elif l == 'H':
            swapped_locations.append('A')
        else:
            swapped_locations.append(l)
    return swapped_locations

In [101]:
def get_games(reg_season, team_id, year):
    current_season = reg_season.loc[(reg_season['Season']==year)]
    current_season = current_season.drop(labels=['Season'], axis=1)
    
    # wins
    team_wins = current_season.loc[(reg_season['WTeamID']==team_id)]
    team_wins['Win'] = 1
    team_wins = team_wins.drop(labels=['WTeamID'], axis=1)
    
    team_wins.columns = ['DayNum', 'Score', 'OppTeam', 'OppScore', 'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF', 'Win']
    team_wins = team_wins[['DayNum', 'OppTeam', 'Loc', 'NumOT', 'Win', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']]
    
    # losses
    team_losses = current_season.loc[(reg_season['LTeamID']==team_id)]
    team_losses['Win'] = 0
    team_losses = team_losses.drop(labels=['LTeamID'], axis=1)

    team_losses['Loc'] = swap_locations(team_losses['WLoc'].values)
    team_losses = team_losses.drop(labels=['WLoc'], axis=1)
    
    team_losses.columns = ['DayNum', 'OppTeam', 'OppScore', 'Score', 'NumOT', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Win', 'Loc']
    team_losses = team_losses[['DayNum', 'OppTeam', 'Loc', 'NumOT', 'Win', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']]
    
    all_games = team_wins.append(team_losses).reset_index(drop=True)
    all_games = standardize_ot(all_games)
    return all_games

In [102]:
def standardize_ot(games):
    for c in stat_categories:
        games[c] = games[c] / ((40 + 5*games['NumOT']) / 40)
    games = games.drop(labels=['NumOT'], axis=1)
    return games

In [103]:
def average_stats(games, team_id):
    stats = games[stat_categories]
    stats = stats.mean(axis=0)
    return stats.to_frame(name=team_id).transpose()

In [104]:
def season_averages(reg_season, year):
    df = pd.DataFrame(columns=stat_categories)
    
    for t in team_ids:
        games = get_games(reg_season, t, year)
        stats = average_stats(games, t)
        df = df.append(stats)
    
    df['TeamID'] = df.index
    df['Season'] = year
    
    return df

In [105]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
df = pd.DataFrame(columns=stat_categories)

for y in years:
    print y
    avg = season_averages(reg_season, y)
    df = df.append(avg, ignore_index=True)

2010


/Users/emilyhedlund/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/emilyhedlund/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


2011
2012
2013
2014
2015
2016
2017
2018


In [110]:
df = df[['Season', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']]
df = df.dropna(axis=0, subset=stat_categories, how='all')
df['Season'] = df['Season'].astype(int)
df['TeamID'] = df['TeamID'].astype(int)
df.to_csv("regular_season.csv", index=False)